In [1]:
import numpy as np
import pandas as pd
import twint
import time
from datetime import datetime
from datetime import timedelta, date
from collections import Counter
from ast import literal_eval

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import nest_asyncio
nest_asyncio.apply()

In [2]:
def clean_reply_to(obs):
    
    if len(obs) != 0:

        return [int(x['id']) for x in (obs)]
            
    
# vec_clean_reply_to = np.vectorize(clean_reply_to)

In [3]:
language_replacement_dict = {'en': 1, 'und':0}

def replace_0_or_1(x):
    if x not in (0,1):
        return np.nan
    else:
        return x

def clean_language(series):
    local_series = series.map(language_replacement_dict)
    
    return [replace_0_or_1(x) for x in local_series]

In [4]:
def clean_tweets(df):
    local_df = df
    
    
    # drop redundant cols
    local_df.drop(['created_at', 'timezone', 'cashtags', 'user_id_str', 'photos', 'video', 'thumbnail', 'translate', 'trans_src', 'trans_dest', 'name', 'search', 'day', 'hour', 'link', 'quote_url'], axis=1, inplace=True)
    
    
    # convert date to datetime
    local_df['date'] = pd.to_datetime(local_df['date'])
    
    
    # clean "reply_to" field
    local_df['reply_to'] = [clean_reply_to(x) for x in local_df['reply_to']]
    
    
    # clean "language field"
    local_df['language'] = clean_language(local_df['language'])
    
    
    return local_df


# Final Loop

In [5]:
#PATH_ROOT = FOLDER WHERE YOU WANT TO SAVE TWEETS

#DELETE THIS:
PATH_ROOT = "C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Looking Up All Conspiracy Hashtag User Tweets/All Conspiracy Tweeters' Tweets/"

In [6]:
def search_for_link(links, date_tuple, SAVE_PATH_ROOT):
    '''
    link = str(web domain)
    '''
    c = twint.Config()
                
    c.Hide_output = True  
    
    c.Search = links
    
    # find shadow-banned accounts too - this apparently slows things down considerably
    c.Profile_full = True    
    
    c.Since = date_tuple[0]
    c.Until = date_tuple[1]

    c.Pandas = True

    twint.run.Search(c)
    
    clean_tweets(twint.storage.panda.Tweets_df).to_csv(fr"{SAVE_PATH_ROOT}/{date_tuple[0]} to {date_tuple[1]}_LINKS.csv", index=False)

## List of CT Links

In [7]:
CT_link_list = ['zerohedge.com', 'infowars.com', 'principia-scientific.com'
'tx.voice-truth.com', 'humansarefree.com', 'activistpost.com'
'gnews.org', 'wakingtimes.com', 'brighteon.com']

CT_links = ' OR '.join(CT_link_list)

## Import date ranges

In [8]:
date_ranges_df = pd.read_csv(r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Getting Conspiracy Hashtag Users/Date Ranges.csv")

date_ranges = [literal_eval(x) for x in date_ranges_df['dates'].to_list()]

# final loop

In [9]:
LINK_SAVE_PATH = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Twint Output - Links"

In [ ]:
for date_tup in date_ranges:
    
    search_for_link(links=CT_links, date_tuple=date_tup, SAVE_PATH_ROOT=LINK_SAVE_PATH)

In [ ]:
# test = pd.read_csv(LINK_SAVE_PATH + "/2020-01-05 to 2020-01-11_LINKS.csv")